In [1]:
import os
from config import *
import torch
from dataloaders import CNNDataset
from torch.utils.data import random_split, DataLoader
import pandas as pd



In [2]:
dataset = CNNDataset(data_csv=P_TRAIN_CSV, target_csv=P_TARGETS_CSV, matrix_dir=ETERNA_PKG_BPP )

(197347,) 
 (197347, 2)
         sequence_id                                               path
134403  09dcbc47a45a  DATA/stanford-ribonanza-rna-folding/Ribonanza_...
134407  3685502b1bb3  DATA/stanford-ribonanza-rna-folding/Ribonanza_...
134408  fcfb7ca3e545  DATA/stanford-ribonanza-rna-folding/Ribonanza_...
134410  28927990459c  DATA/stanford-ribonanza-rna-folding/Ribonanza_...
134412  a0d0e663cc20  DATA/stanford-ribonanza-rna-folding/Ribonanza_...


In [3]:
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [0.7, 0.3], generator1)

In [4]:
dataset.matrix_df[dataset.matrix_df['sequence_id'] == '00257e85caac']['path']

143378    DATA/stanford-ribonanza-rna-folding/Ribonanza_...
Name: path, dtype: object

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, persistent_workers=True)  
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, persistent_workers=True) 

In [ ]:
dataset.__getitem__(0)

In [6]:
for x, y in train_dataloader:
    print(x,y)
    break

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/Mina/miniconda3/envs/rna/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/Mina/miniconda3/envs/rna/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "/Users/Mina/miniconda3/envs/rna/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 364, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "/Users/Mina/miniconda3/envs/rna/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 364, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "/Users/Mina/workspace/RNA_Reactivity_Prediction/dataloaders.py", line 56, in __getitem__
    self.reactivity = self.transform(self.reactivity)
  File "/Users/Mina/m

In [3]:
targets = pd.read_csv(P_TARGETS_CSV)

In [12]:
targets.iloc[0, :-1].values

array([1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       -0.036, 0.683, 0.189, -0.005, -0.039, -0.017, 0.027, -0.007, 0.148,
       0.028, -0.144, -0.061, -0.073, 0.009, 0.0, 0.028, -0.034, -0.109,
       -0.256, 0.078, 0.284, 0.361, 0.631, -0.027, -0.029, 0.076, 0.515,
       0.169, 0.2, 0.012, 0.263, 0.477, 0.531, 2.283, 0.981, 2.821, 0.155,
       -0.191, 0.003, 0.02, 0.414, 0.262, 0.186, 1.138, -0.157, 0.085,
       -0.04, -0.008, -0.077, -0.104, -0.079, -0.041, 0.193, -0.012,
       0.018, 0.448, -0.044, 0.303, -0.064, 0.063, -0.007, -0.205, -0.065,
       -0.097, -0.026, -0.173, -0.036, -0.329, 0.11, 0.0, 0.002, 0.282,
       0.231, 0.282, 0.123, -0.068, 0.105, -0.005, -0.128, -0.103, 0.093,
       -0.027, -0.108, 0.112, -0.053, -0.097, -0.073, 0.088, 0.049, 0.0,
       -0.015, 0.01, 0.009, 0.0, 0.313, 0.329, 0.823, 1.813, 0.484, 0.611,
       0.524, 0.453, 0.566, 0.0, 0.0, 0.0, 0.0, 

In [13]:
test = 'DATA/Ribonanza_bpp_files/extra_data/9/8/2/ff4dcd9bf671.txt'
import numpy as np

# Load the data from the file
data = np.loadtxt(test)

# Determine the shape of the array
max_row = int(data[:, 0].max())
max_col = int(data[:, 1].max())

# Create an empty array filled with zeros
filled_array = np.zeros((max_row, max_col))

# Fill the values from the loaded data into the empty array
for row, col, value in data:
    filled_array[int(row) - 1, int(col) - 1] = value

In [15]:
filled_array.shape

(191, 207)

In [40]:
dara = pd.read_csv(P_TRAIN_CSV)

In [30]:

file_list = []
file_paths = []
for root, dirs, files in os.walk(ETERNA_PKG_BPP):
    for file in files:
        if file.endswith('.txt'):
            file_list.append(os.path.join(root, file))
            file_paths.append(file[:-4])


In [31]:
matrix_df = pd.DataFrame(columns=['sequence_id', 'path'])
matrix_df['sequence_id'] = file_paths
matrix_df['path'] = file_paths

In [39]:
len(set(matrix_df['sequence_id'].to_list()) - set(targets['sequence_id'].to_list()))

1915372

In [ ]:
matrix_df.drop(matrix_df['sequence_id']!=targets['sequence_id'])

In [ ]:
matrix_path = dataset.matrix_df[dataset.matrix_df['sequence_id'] == '00257e85caac']['path'][0]


In [20]:
matrix_path

str